In [95]:
!pip install "tensorflow-text==2.8.*"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [96]:
import pandas as pd
import numpy as np
import re, string
import os 
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import linear_kernel

In [97]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
af_center = pd.read_csv("/content/drive/MyDrive/Fake news/News-Dataset/ภาษาไทย_ศูนย์ต่อต้านข่าวปลอม/dataset-antifakenewscenter.csv")
sure_share = pd.read_csv("/content/drive/MyDrive/Fake news/News-Dataset/Sure and Share/sure_and_share.csv")

In [99]:
df_news = pd.concat([af_center, sure_share])
df_news = df_news.drop(['Unnamed: 0'], axis=1).reset_index(drop=True)
df_news

,headline,url,img_src,tag,topic
0,ข่าวปลอม อย่าแชร์! เพจ Facebook เชิญชวนลงทุนให...,https://www.antifakenewscenter.com/news/activi...,https://www.antifakenewscenter.com/wp-content/...,ข่าวปลอม,การเงิน-หุ้น
1,ข่าวปลอม อย่าแชร์! การขึ้นเงินรางวัลสลากดิจิทั...,https://www.antifakenewscenter.com/news/activi...,https://www.antifakenewscenter.com/wp-content/...,ข่าวปลอม,นโยบายรัฐบาล-ข่าวสาร
2,ไทยเพิ่มโรคฝีดาษลิง เป็นโรคติดต่อเฝ้าระวังลำดั...,https://www.antifakenewscenter.com/news/activi...,https://www.antifakenewscenter.com/wp-content/...,ข่าวจริง,นโยบายรัฐบาล-ข่าวสาร
3,ข่าวปลอม อย่าแชร์! ธปท. ส่งเอกสารแจ้งผู้ขอกู้เ...,https://www.antifakenewscenter.com/news/activi...,https://www.antifakenewscenter.com/wp-content/...,ข่าวปลอม,การเงิน-หุ้น
4,ข่าวปลอม อย่าแชร์! ผลิตภัณฑ์ Lutiana VIS ช่วยบ...,https://www.antifakenewscenter.com/news/activi...,https://www.antifakenewscenter.com/wp-content/...,ข่าวปลอม,ผลิตภัณฑ์สุขภาพ
...,...,...,...,...,...
6298,ชัวร์ก่อนแชร์ : แว่นตามแผงลอยแก้ปัญหาสายตาได้จ...,https://www.youtube.com/watch?v=uha4ebTV3J0,https://img.youtube.com/vi/uha4ebTV3J0/0.jpg,NaN,NaN
6299,ชัวร์ก่อนแชร์ : แบตเตอรี่มีอายุการใช้งานเพียง ...,https://www.youtube.com/watch?v=ximdyuk9B-g,https://img.youtube.com/vi/ximdyuk9B-g/0.jpg,NaN,NaN
6300,ชัวร์ก่อนแชร์ : 5 เทคนิคยืดอายุเกียร์ออโต้ ใช้...,https://www.youtube.com/watch?v=Yo-E8nlt7Y4,https://img.youtube.com/vi/Yo-E8nlt7Y4/0.jpg,NaN,NaN
6301,ชัวร์ก่อนแชร์ : ไมโครเวฟอันตรายจริงหรือ?,https://www.youtube.com/watch?v=_yfjR-51n8A,https://img.youtube.com/vi/_yfjR-51n8A/0.jpg,NaN,NaN


In [100]:
df_news.to_csv('/content/drive/MyDrive/Fake news/News-Dataset/df_news.csv')

In [101]:
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3' #@param ['https://tfhub.dev/google/universal-sentence-encoder-multilingual/3', 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3']

model = hub.load(module_url)

In [102]:
def embed(input):
    return model(input)

In [103]:
Model_USE= embed(df_news.headline)

In [104]:
exported = tf.train.Checkpoint(v=tf.Variable(Model_USE))
exported.f = tf.function(
    lambda  x: exported.v * x,
    input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])
tf.saved_model.save(exported,'TrainUSE')

INFO:tensorflow:Assets written to: TrainUSE/assets


INFO:tensorflow:Assets written to: TrainUSE/assets


In [105]:
imported = tf.saved_model.load('TrainUSE')
loadedmodel =imported.v.numpy()

In [106]:
!pip install spacy-universal-sentence-encoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [107]:
import spacy_universal_sentence_encoder
nlp = spacy_universal_sentence_encoder.load_model('xx_use_lg')

In [108]:
def SearchDocument(query):
    q =query
    # embed the query for calcluating the similarity
    Q_Train =[nlp(q).vector]
    
    #imported_m = tf.saved_model.load('/home/zettadevs/GoogleUSEModel/TrainModel')
    #loadedmodel =imported_m.v.numpy()
    # Calculate the Similarity
    linear_similarities = linear_kernel(Q_Train, loadedmodel).flatten() 
    #Sort top 10 index with similarity score
    Top_index_doc = linear_similarities.argsort()[:-11:-1]
    # sort by similarity score
    linear_similarities.sort()
    a = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        a.loc[i,'index'] = str(index)
        a.loc[i,'headline'] = df_news['headline'][index] ## Read File name with index from File_data DF
        try:
          a.loc[i,'tag'] = df_news['tag'][index]
        except:
          pass
        a.loc[i,'url'] = df_news['url'][index]
        a.loc[i,'img_src'] = df_news['img_src'][index]
    for j,simScore in enumerate(linear_similarities[:-11:-1]):
        a.loc[j,'Score'] = simScore
    return a

In [109]:
search_result = SearchDocument('ชัวร์ก่อนแชร์. สนใจเช็คข่าวนี้มั๊ยคะ. // .รบกวนบอกบุญนะครับ. ช่วย ๆ กันอีกไม่เกิน 3 เดือน" .....วัดพระบาทน้ำพุ ต้อง ..ปิดลง!!(ถ้าไม่อ่าน) ” ขอทานน้ำใจ “ 1 แชร์ 1 ธรรมทาน” ... “ 15 บาท ของท่าน ช่วยผู้ป่วยโรคเอดส์ " ทุกคนในนี้มีมือถือใช่ไหมครับ ".....ขอซัก 15 บาทได้ไหม เพื่อผู้ป่วยโรคเอดส์ ไม่ว่ามือถือคุณจะเครื่องละพัน หรือเครื่องละหมื่น ขอแค่ 15 บาท... " บริจาคให้วัดพระพุทธบาตรน้ำพุ "...ง่ายๆ แค่กด 1900-222-200 และกด 1 แค่นี้ คุณก็...ได้ร่วมทำบุญแล้วค่ะ เรื่องจริง โทรไปเช็คที่วัดแล้ว) ..ขออนุโทนา สำหรับทุกสายทานบารมี อนุโมทนา ด้วยนะครับ ..... " อีกไม่เกิน 3 เดือน" .....วัดพระบาทน้ำพุต้อง" ...ปิดลง!! "... ถ้าไม่อ่าน กรุณา Forward.ถ้าไม่ช่วย อีกไม่ เกิน 3 เดือน วัดพระบาทน้ำพุ ต้องปิดลง ทุกคนคงเคยได้ยินเรื่องราวเกี่ยวกับวัดพระบาทน้ำพุ จังหวัดลพบุรี ที่มีหลวงพ่ออลงกตเป็นเจ้าอาวาส ซึ่งท่านได้อุทิศตัวช่วยเหลือผู้ป่วยโรคเอดส์และเด็กกำพร้ามาสิบกว่าปี แล้ว ทั้งๆที่ท่านมีพร้อม ทุกอย่าง จบการ ศึกษาระดับปริญญาโทวิศวะจากออสเตรเลีย แต่ท่านก็เสียสละ ได้ เพียงเพราะท่านเห็น ว่า ผู้ป่วยโรคเอดส์นั้นไร้ที่พึ่ง จริงๆ ขนาดบางคนพอพ่อแม่รู้ว่าติดเชื้อ เอดส์ ยังรังเกียจและทอดทิ้งลูกของตัว เองได้เลย หลวงพ่อท่านเห็นว่า ถ้าท่านไม่ช่วยพวกเขาเหล่า นี้ ท่านก็ไม่สามารถเรียกตัวเองว่าเป็น มนุษย์ได้ ทุก วันนี้ ที่วัดมีผู้ป่วยและเด็กกำพร้าที่ หลวงพ่อต้องคอยดูแลรวมถึงพนักงานและอาสาสมัครราวหนึ่งพันคน ค่าใช้จ่ายต่อเดือนประมาณ สามล้าน กว่าบาทแต่ยอดบริจาคกลับน้อยลง เนื่องจากสภาวะเศรษฐกิจที่ซบเซา เห็นว่า ลดลงเหลือเพียงเดือนละสองแสนบาทเอง ส่วนรัฐบาลก็ช่วยเหลือเพียงเดือนละหนึ่งแสนบาทเท่า นั้นเคยโทรไปถามที่วัด เกี่ยวกับสถานะทางการเงิน พนักงานก็บอกว่า รายรับเท่าเดิม แต่ค่าใช้จ่ายเพิ่มขึ้นทุกวัน ค่ายารักษาหรือเพิ่มภูมิต้านทานก็ แสนจะแพง แถมผู้ป่วยก็มีแต่จะเพิ่มมาก ขึ้น ทางวัดจะไม่รับก็ไม่ ได้ เนื่องจากมีคนพาผู้บ่วยมาทิ้งไว้ที่ หน้าประตูวัดเสมอ ซึ่งพวกเขาก็ไม่มีทางไป ที่วัดก็เมตตาช่วยเหลือแม้กระทั่งคน ชราและเด็กที่คนในครอบครัวเสียชีวิต เพราะเอดส์แล้วไม่มีใครดูแล ตอนนี้ต้องมีการส่ง ผู้ป่วยที่อาการดีแล้วและพอมีฐานะกลับบ้านบ้าง แล้ว และอีกสามเดือนอาจต้องปิดตัว ลง!!!!!!หลวงพ่อเองต้องลงมาบิณฑบาตรที่ กรุงเทพฯทุกสัปดาห์ ต้องไปหลายที่ต่อ หนึ่งวัน เพราะรอนไปช่วยเหลือถึงวัด ไม่ไหว เห็นแล้วก็เหนื่อยแทน จริงๆ เมืองไทยมีผู้ติดชื้อเอดส์มาก เป็นอันดับสี่ของโลกแล้ว และแนวโน้มก็มี มากขึ้นเรื่อยๆ ส่วนทางกับอายุของผู้ที่ ติดเชื้อเอดส์ซึ่งกลับกลายเป็นว่าอายุน้อยลง ทุกที อยากให้พวกเราเข้าใจว่า เรื่องนี้เป็นปัญหาของประเทศชาติและ เป็นเรื่องของพวกเราทุกคนที่ต้องช่วยกัน พวกเราสามารถช่วยได้หลายรูปแบบขณะนี้หลวงพ่อท่านมารับบริจาคทุกวัน เสาร์ ตั้งแต่ 8.30 น. -10.00 น. ที่สวน ลุมไนท์บาร์ซาร์ ข้างๆอาคารบีอีซี เท โร สามารถบริจาคเป็นเงิน (ดีที่สุด), ยา, ผ้าอ้อม, สำลี, ของอุปโภคบริโภคต่างๆ, หนังสือ, เสื้อผ้า ฯลฯหรือบริจาคผ่านธนาคาร ให้กับ  กองทุนอาทรประชานาถ ถ้าสามารถทำเป็นรายเดือนได้จะดีมาก ราย ละเอียดของเบอร์บัญชีมีดังนี้ ธ.กรุงเทพฯ สาขา ลพบุรี 289-0-84697-1ธ.ทหารไทย สาขา ลพบุรี 304-2-41277-9ธ.กสิกรไทย สาขา ถนนสุรสงคราม 174-2-39000-0ธ.ไทย พาณิชย์ สาขาลพบุรี 579-2-33730- 7ธ.กรุงศรีอยุธยา สาขา ลพบุรี 111-1-47300-7ธ.นครหลวง ไทย สาขาลพบุรี 340-2-14976- 0ธ.เพื่อการเก ษตรและสหกรณ์การเกษตร สาขานางเลิ้ง 000-2-12022-0และสุดท้ายที่พวกเราสามารถช่วย ได้เดี๋ยวนี้ คือ การบอกต่อถึงเรื่องนี้ และช่วยออส่งเมล์นี้ไปให้คนที่รู้จักทกๆ คน..ฝันส่งมาค่ะ')

In [110]:
search_result = search_result[search_result['Score']>=0.4]
search_result

,index,headline,tag,url,img_src,Score


In [111]:
cards = []
for i in range(search_result.shape[0]):
        card = {
            "thumbnailImageUrl": search_result.iloc[i,:].img_src,
            "imageBackgroundColor": "#FFFFFF",
            "title": search_result.iloc[i,:].headline,
            "text": search_result.iloc[i,:].tag,
            "defaultAction": {
                "type": "uri",
                "label": "ดูเพิ่มเติม",
                "uri": search_result.iloc[i,:].url,
            }
        }
        cards.append(card)
response = {
        "fulfillmentMessages": [
      {
        "payload": {
            "line": {
              "type": "template",
              "altText": "this is a carousel template",
              "template": {
                "type": "carousel",
                "columns": cards,
                "imageAspectRatio": "rectangle",
                "imageSize": "cover"
              }
            }
          }
      }
    ],
        #"fulfillmentText": res,
        "source": "webhook"
    }

In [112]:
cards

[]